## Import statements

In this notebook, we compare various heterogeneity-agnostic and heterogeneity-aware FIFO policies, with and without space sharing, on traces with single-GPU and multi-GPU jobs.

In [ ]:
from plotting_utils import plot_metric_vs_inverse_lambda, plot_jct_cdf
from utils import get_logfile_paths, average_jct_fn

## Get list of relevant logfiles and define label mapping

In [ ]:
logfile_paths = sorted(get_logfile_paths(
    "/future/u/deepakn/gavel/logs/single_gpu_continuous_jobs_final/"))

In [ ]:
labels = {"fifo": "FIFO",
          "fifo_perf": "Gavel",
          "fifo_packed": "Gavel w/ SS"}

## Plot average JCT vs. input job rate


In [ ]:
plot_metric_vs_inverse_lambda(
    logfile_paths,
    labels,
    v100s=36, p100s=36, k80s=36,
    policies=["fifo",
              "fifo_perf",
              "fifo_packed"],
    metric_fn=lambda x: average_jct_fn(x, min_job_id=4000, max_job_id=5000),
    metric_label="Average JCT\n(hours)",
    xmax=2.8,
    ymax=100.0,
    extrapolate=True,
    output_filename="continuous_jobs/multi_gpu_fifo_v100=36_p100=36_k80=36.pdf"
)